In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import os
os.chdir('C:/Users/andre/github/algoritmos-python/')
from mlkit.nlp.pre_process.clean_text import TextCleaner
import re

In [ ]:
! pip install spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download en

In [2]:
def score(X, x, c = 2):
    m = X.mean(0)
    N = x.shape[0]
    alfa = c * m
    beta = c * (1 - m)
    alfa_ = alfa + x.sum(0)
    beta_ = beta + N - x.sum(0)
    nc = (np.log(alfa + beta) - np.log(alfa + beta + N) + np.log(beta_) - np.log(beta)).sum(1)
    q = np.log(alfa_) - np.log(alfa) + np.log(beta) - np.log(beta_)
    s = nc + (X * q.T)
    
    return s

In [5]:
def search(query, corpus, clean_text = True):    
    # clean text
    if clean_text:
        cleaner = TextCleaner(clean_regex = ".*")
        query = cleaner.clean(query)
        corpus = cleaner.clean(corpus)
    
    # create DTM
    vec = CountVectorizer().fit(corpus)
    DTM = vec.transform(corpus)
    DTM_query = vec.transform(query)
    
    # calculate scores
    s = score(DTM, DTM_query)
    
    return s

In [6]:
df = pd.read_csv('~/github/data/datasets/movies.csv')

# remove release year from movie title
df['title'] = df['title'].apply(lambda x: re.sub("\\([0-9]+\\)", "", x))

s = search(['toy story', 'the lion king','alladin','beauty and the beast','cinderella','little mermaid','hercules'] , df['title'])

data = {'title': df['title'], 'value': np.asarray(s).tolist()}
df = pd.DataFrame(data)
df['value'] = df['value'].apply(lambda x: x[0])
df.sort_values('value', ascending = False).iloc[0:30, :]

,title,value
1997,"Little Mermaid, The",10.091983
23240,Beauty and the Beast,9.963770
589,Beauty and the Beast,9.963770
23241,Beauty and the Beast (Beauty and the Beasts: A...,9.944825
360,"Lion King, The",9.696858
0,Toy Story,9.044807
3027,Toy Story 2,9.044807
15401,Toy Story 3,9.044807
7960,"Cinderella Story, A",9.044807
9398,"Lion King 1½, The",8.192724
